In [20]:
import pandas as pd
import numpy as np
import nltk
import re
import os
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
import datetime
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

In [2]:
#load sample file
example_file = "./tweets/J.P. Morgan.csv"
tweets_df = pd.read_csv(example_file, index_col=0)
print(tweets_df.iloc[0,0])
tweets_df

“The vaccine news and the focus on that is the most important thing for the market at the moment,” said Stephanie R… https://t.co/G8PdiRLkVt


,0
2020-12-08 23:51:00,“The vaccine news and the focus on that is the...
2020-12-08 23:30:03,The J.P. Morgan Healthcare #Conference is goin...
2020-12-08 23:17:45,RT @NathaniNoShame: every night in fear that s...
2020-12-08 23:10:42,Alphabet Inc. (GOOG) surprised the market with...
2020-12-08 23:07:06,@TheSolariReport Millionaires don't use Astrol...
...,...
2020-11-30 00:41:31,The Liquidating Team in play.\nFirst the manip...
2020-11-30 00:26:19,"J P Morgan, Ghost of Rich Dude's Past"
2020-11-29 23:42:38,"Go as far as you can see; when you get there, ..."
2020-11-29 23:05:40,"Had a great morning at BWW, with my son eating..."


In [3]:
#remove stopword
#remove punctuations
#remove special characters
#do we need to stem the words? 
#tokenize

In [4]:
def remove_emoji(text):
    emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  # emoticons
    u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
    u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
    u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
    u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
    "+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', text)

def preprocess(sentence):
    special_marks = ['&', '\n', '@', '$', '#', '☆', '!', '(', ')'
                     ',', '.', ';', ':', '💰', '✅']
    
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence = re.sub('@[^, ]*', '', sentence) #remove twitter user name after @
    for sm in special_marks:
            sentence = sentence.replace(sm, "")
            
    sentence = sentence.replace('{html}',"") 
    sentence = remove_emoji(sentence)
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    #stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
    return " ".join(lemma_words)

def preprocess_tweet(file):
    tweets_df = pd.read_csv(file, index_col=0)
    # drop duplicate
    tweets_df.drop_duplicates(inplace=True)
    
    tweets_df['cleanText']=tweets_df['0'].map(lambda s:preprocess(s)) 
    
    return tweets_df

tweets_clean = preprocess_tweet(example_file)

In [5]:
tweets_clean

,0,cleanText
2020-12-08 23:51:00,“The vaccine news and the focus on that is the...,vaccine news focus important thing market mome...
2020-12-08 23:30:03,The J.P. Morgan Healthcare #Conference is goin...,morgan healthcare conference going virtual exp...
2020-12-08 23:17:45,RT @NathaniNoShame: every night in fear that s...,every night fear someone knew would kill liter...
2020-12-08 23:10:42,Alphabet Inc. (GOOG) surprised the market with...,alphabet inc goog surprised market result morg...
2020-12-08 23:07:06,@TheSolariReport Millionaires don't use Astrol...,millionaire use astrology billionaire dojp morgan
...,...,...
2020-11-30 00:48:47,@John11019 he was deceived by J.P. Morgan and ...,deceived morgan thomas edison
2020-11-30 00:41:31,The Liquidating Team in play.\nFirst the manip...,liquidating team playfirst manipulation electi...
2020-11-30 00:26:19,"J P Morgan, Ghost of Rich Dude's Past",morgan ghost rich dude past
2020-11-29 23:05:40,"Had a great morning at BWW, with my son eating...",great morning bww son eating wing parking lot ...


## Use Textblob to classify tweets

In [6]:
from datetime import datetime

#use textbolb to get sentiment scores
from textblob import TextBlob
#https://textblob.readthedocs.io/en/dev/
def get_sentiment(tweet):
    analysis = TextBlob(tweet) 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'
        
def classify_sentiment(tweets_clean):
    sentiment = []
    for index, row in tweets_clean.iterrows(): 
        sentiment.append(get_sentiment(row['cleanText']))
    tweets_clean['sentiment'] = sentiment
    tweets_clean.index = [datetime.strptime(x, "%Y-%m-%d %H:%M:%S") \
                          for x in \
                          tweets_clean.index]
    
    return tweets_clean

tweets_classified = classify_sentiment(tweets_clean)
tweets_classified = tweets_classified[::-1]
tweets_classified

,0,cleanText,sentiment
2020-11-29 22:27:13,J.P. Morgan: iPhone 12 Pro reaches record lead...,morgan iphone pro reach record lead time,neutral
2020-11-29 23:05:40,"Had a great morning at BWW, with my son eating...",great morning bww son eating wing parking lot ...,positive
2020-11-30 00:26:19,"J P Morgan, Ghost of Rich Dude's Past",morgan ghost rich dude past,positive
2020-11-30 00:41:31,The Liquidating Team in play.\nFirst the manip...,liquidating team playfirst manipulation electi...,neutral
2020-11-30 00:48:47,@John11019 he was deceived by J.P. Morgan and ...,deceived morgan thomas edison,neutral
...,...,...,...
2020-12-08 23:07:06,@TheSolariReport Millionaires don't use Astrol...,millionaire use astrology billionaire dojp morgan,neutral
2020-12-08 23:10:42,Alphabet Inc. (GOOG) surprised the market with...,alphabet inc goog surprised market result morg...,positive
2020-12-08 23:17:45,RT @NathaniNoShame: every night in fear that s...,every night fear someone knew would kill liter...,neutral
2020-12-08 23:30:03,The J.P. Morgan Healthcare #Conference is goin...,morgan healthcare conference going virtual exp...,neutral


In [7]:
print('# of positive tweets: ',(tweets_classified['sentiment']=='positive').sum())
print('# of negative tweets: ',(tweets_classified['sentiment']=='negative').sum())
print('# of neutral tweets: ',(tweets_classified['sentiment']=='neutral').sum())

# of positive tweets:  398
# of negative tweets:  100
# of neutral tweets:  335


## Get Stock data

In [8]:
JPM_price = pd.read_excel("./stockdata/JPM.xlsx").set_index('Date')
SPY_price = pd.read_excel("./stockdata/SPY.xlsx").set_index('Date')#benchmark
print(len(JPM_price),len(SPY_price))

828 827


In [66]:
#calculate Excess of Log-return

def excess_log_return(df,SPY_price, freq='H'):
    SPY_price= SPY_price.iloc[::-1]
    SPY_price.rename(columns={"Last Price": "SPY_price", 
                              "Volume": "SPY_vol",
                              "SMAVG (15)":"SPY_SMAVG"},inplace = True)
    df = df.iloc[::-1] 
    df_all = pd.concat([SPY_price,df], axis=1, join='inner').reindex(SPY_price.index)
    #use specific time interval, e.g every hour
    times = pd.date_range(start='2020-11-23 09:30:00', end='2020-12-08 15:55:00', freq=freq)
    df_all['filter'] = [True if x in times else False for x in df_all.index]
    df_all = df_all[df_all['filter']]
    #calclulate log return
    df_all['log_return'] = np.log(df_all['Last Price']) - np.log(df_all['Last Price'].shift(1))
    df_all['SPY_log_return'] = np.log(df_all['SPY_price']) - np.log(df_all['SPY_price'].shift(1))
    df_all['excess_log_return'] = df_all['log_return'] - df_all['SPY_log_return']
    
    #calculate rolling volatility, starting date = 2020-11-30 09:30:00, price starting date: 2020-11-23 09:30:00
    window = 3 * 7 + 4 # 7 days * 7 hours per day: 9:30 - 15:30
    dpy = 252 * 7  # trading hours per year
    ann_factor = dpy / window  
    df_all['volatility'] = df_all['log_return'].rolling(window).std() * ann_factor #annulized volatility
    
    return df_all.loc[:,['Last Price','Volume','log_return','excess_log_return','volatility']]

df_JPM = excess_log_return(JPM_price,SPY_price)    
df_JPM.head(40)

,Last Price,Volume,log_return,excess_log_return,volatility
Date,,,,,
2020-11-23 09:30:00,115.7200,579903,NaN,NaN,NaN
2020-11-23 10:30:00,116.8400,149744,0.009632,0.012798,NaN
2020-11-23 11:30:00,116.7000,135184,-0.001199,0.002709,NaN
2020-11-23 12:30:00,117.0000,72067,0.002567,-0.000756,NaN
2020-11-23 13:30:00,117.1100,54381,0.000940,-0.000402,NaN
2020-11-23 14:30:00,117.4550,87122,0.002942,0.002465,NaN
2020-11-23 15:30:00,118.0150,147385,0.004756,0.003034,NaN
2020-11-24 09:30:00,120.4500,1072831,0.020423,0.014134,NaN
2020-11-24 10:30:00,122.0300,155480,0.013032,0.010533,NaN


In [15]:
#把twitter和price的frequency对上, 然后计算每小时的sentiment score
def cal_sentiment_scores(df_price, tweets_classified):
    tweets_classified.index = pd.to_datetime(tweets_classified.index)
    tweets_count = [None]
    sentiment_abs_score = [None]
    sentiment_rela_score = [None]
    df_price = df_price[df_price.index > tweets_classified.index[0]]
    for i in range(len(df_price)-1):
        #todo: 把开头多余的stock 天数去掉
        temp = tweets_classified[(tweets_classified.index > \
                                  df_price.index[i]) \
                                 & (tweets_classified.index < \
                                    df_price.index[i+1])]
        tweets_count.append(len(temp))
        pos_num = len(temp[temp['sentiment']=='positive'])
        neg_num = len(temp[temp['sentiment']=='negative'])

        sentiment_abs_score.append(pos_num - neg_num)
        try:
            sentiment_rela_score.append((pos_num - neg_num)/(pos_num + neg_num))
        except:
            sentiment_rela_score.append(None) #如果分母为0

    df_price['tweets_num'] = tweets_count
    df_price['senti_abs_score'] = sentiment_abs_score
    df_price['senti_rela_score'] = sentiment_rela_score
    return df_price
    
df_JPM_senti = cal_sentiment_scores(df_JPM, tweets_classified)
df_JPM_senti

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

,Last Price,Volume,log_return,excess_log_return,tweets_num,senti_abs_score,senti_rela_score
Date,,,,,,,
2020-11-30 09:30:00,120.5700,396875,-0.005542,-0.005459,NaN,NaN,NaN
2020-11-30 10:30:00,119.0900,119408,-0.012351,-0.006297,0.0,0.0,NaN
2020-11-30 11:30:00,118.6800,89106,-0.003449,-0.000589,1.0,1.0,1.000000
2020-11-30 12:30:00,118.6600,66828,-0.000169,-0.003888,2.0,2.0,1.000000
2020-11-30 13:30:00,118.7000,51898,0.000337,0.000171,10.0,3.0,0.600000
2020-11-30 14:30:00,118.6000,75479,-0.000843,-0.001784,9.0,2.0,1.000000
2020-11-30 15:30:00,118.5300,100082,-0.000590,-0.001088,8.0,0.0,0.000000
2020-12-01 09:30:00,119.8150,423108,0.010783,0.001231,60.0,17.0,0.485714
2020-12-01 10:30:00,120.7159,121895,0.007491,0.003363,3.0,2.0,1.000000


In [30]:
stock_file_list = os.listdir("./stockdata")
stock_file_list

['ALGN.xlsx',
 'AMP.xlsx',
 'AON.xlsx',
 'BRK.B.xlsx',
 'CMCSA.xlsx',
 'CMI.xlsx',
 'CNP.xlsx',
 'DG.xlsx',
 'EXR.xlsx',
 'JKHY.xlsx',
 'JPM.xlsx',
 'LHX.xlsx']

## 以下没用
## Get positive & Negative word count

In [6]:
pos_list = list(pd.read_csv('./opinion-lexicon-English/positive-words.txt',header = None)[0])
neg_list = list(pd.read_csv('./opinion-lexicon-English/negative-words.txt',
                            header = None,encoding='latin-1')[0])

In [7]:
#record positive and negative word counts

def count_words(tweets_clean):
    #count the number of positive and negative words. Read in dataframe with column['cleanText']
    pos_count_list = []
    neg_count_list = []

    for index, row in tweets_clean.iterrows(): 
        pos_count = 0
        neg_count = 0 
        for x in row['cleanText']:
            if x in pos_list:
                pos_count += 1
            if x in neg_list:
                neg_count += 1
        pos_count_list.append(pos_count)
        neg_count_list.append(neg_count)


    tweets_clean['pos_count'] = pos_count_list
    tweets_clean['neg_count'] = neg_count_list
    tweets_clean['word_count'] = [len(x) for x in tweets_clean.cleanText]
    tweets_clean['pct_pos'] = tweets_clean['pos_count']/tweets_clean['word_count']
    tweets_clean['pct_neg'] = tweets_clean['neg_count']/tweets_clean['word_count']
    return tweets_clean

tweets_word_count = count_words(tweets_clean)

In [8]:
tweets_word_count

,0,cleanText,pos_count,neg_count,word_count,pct_pos,pct_neg
2020-12-08 23:51:00,“The vaccine news and the focus on that is the...,"[vaccine, news, focus, important, thing, marke...",1,0,9,0.111111,0.000000
2020-12-08 23:30:03,The J.P. Morgan Healthcare #Conference is goin...,"[morgan, healthcare, conference, going, virtua...",0,0,8,0.000000,0.000000
2020-12-08 23:17:45,RT @NathaniNoShame: every night in fear that s...,"[every, night, fear, someone, knew, would, kil...",0,2,12,0.000000,0.166667
2020-12-08 23:10:42,Alphabet Inc. (GOOG) surprised the market with...,"[alphabet, inc, goog, surprised, market, resul...",0,0,12,0.000000,0.000000
2020-12-08 23:07:06,@TheSolariReport Millionaires don't use Astrol...,"[millionaire, use, astrology, billionaire, doj...",0,0,6,0.000000,0.000000
...,...,...,...,...,...,...,...
2020-11-30 00:48:47,@John11019 he was deceived by J.P. Morgan and ...,"[deceived, morgan, thomas, edison]",0,0,4,0.000000,0.000000
2020-11-30 00:41:31,The Liquidating Team in play.\nFirst the manip...,"[liquidating, team, playfirst, manipulation, e...",0,2,9,0.000000,0.222222
2020-11-30 00:26:19,"J P Morgan, Ghost of Rich Dude's Past","[morgan, ghost, rich, dude, past]",1,0,5,0.200000,0.000000
2020-11-29 23:05:40,"Had a great morning at BWW, with my son eating...","[great, morning, bww, son, eating, wing, parki...",1,0,12,0.083333,0.000000


In [24]:
(tweets_word_count['pct_neg']>0.2).sum()

12

In [25]:
(tweets_word_count['pct_pos']>0.2).sum()

43

In [265]:
df = pd.read_csv('./tweets/Berkshire Hathaway OR BRK.B.csv')

In [266]:
df

,2020-12-11 01:51:59,@DolphfredC See you @ 25 PPS for $NLOK. Consumer cybersecurity has several revenue and cost synergies with many ind… https://t.co/T10vTGtEiG
0,2020-12-11 01:45:38,Blessings!\n\nR.J. Boyd\nLicensed VA Realtor\n...
1,2020-12-11 01:44:36,Another reason why companies like $AAPL and $K...
2,2020-12-11 01:29:08,@tradercracks Berkshire Hathaway
3,2020-12-11 01:22:36,What's your favorite part of Hanukkah?\nBarbar...
4,2020-12-11 00:57:31,Try out these home decor ideas for the most st...
...,...,...
2274,2020-12-01 14:54:20,Letter to shareholders 1990 from Berkshire Hat...
2275,2020-12-01 14:53:12,Tesla's Market Cap Passes Berkshire Hathaway's...
2276,2020-12-01 14:50:59,Get the highest price for your home! Find out ...
2277,2020-12-01 14:46:21,“Beware of geeks bearing formulas.\n—WARREN BU...
